<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_EmotionQADataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_dataset_Medmcqa.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_Emotion"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [5]:
from datasets import load_dataset
print("Preprocessing dataset Emotion")
dataset = load_dataset("dair-ai/emotion")

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["validation"].to_json("validation_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Preprocessing dataset Emotion


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

237173

In [6]:
from datasets import load_dataset
dataset_test = load_dataset("json", data_files="/content/test_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset_test[10] ## empty exp for testing dataset

{'text': 'i don t feel particularly agitated', 'label': 4}

In [8]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")

In [9]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [11]:
nrec=10
dataset_final_Question=dataset['text'][0:nrec]
dataset_final_Answer=dataset['label'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [12]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame

datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [13]:
dataset[10]

{'text': 'i don t feel particularly agitated', 'label': 4}

In [14]:
datasetF

,Question,Answer
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
5,i cant walk into a shop anywhere where i do no...,4
6,i felt anger when at the end of a telephone call,3
7,i explain why i clung to a relationship with a...,1
8,i like to have the same breathless feeling as ...,1
9,i jest i feel grumpy tired and pre menstrual w...,3


In [15]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")

In [16]:
eval_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [17]:
nrec=10
eval_dataset[nrec]

{'text': 'i don t feel particularly agitated', 'label': 4}

In [19]:
eval_dataset[nrec]['text']

'i don t feel particularly agitated'

In [20]:
eval_dataset[nrec]['label']

4

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [51]:
from random import randint
rec = randint(0, len(eval_dataset))

rec = 6

In [52]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rec]['text']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [53]:
outputs

[{'generated_text': 'i felt anger when at the end of a telephone call i was left with the feeling that i had been ignored [/INST] 3 '}]

In [54]:
oanswer=eval_dataset[rec]['label']
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
print(ganswer)
#rand_idx = randint(0, len(eval_dataset))

print(f"Question:\n{eval_dataset[rec]['text']}")
print()
print(f"Original Answer:\n{oanswer}")
print()
print(f"Generated Answer:\n{ganswer}")

from difflib import SequenceMatcher

  # Utility function to compute similarity
def similar(str1, str2):
  return SequenceMatcher(None, str1, str2).ratio()

# using SequenceMatcher.ratio()

qc=str(ganswer).find('[/INST]')
ganswer=ganswer[qc+8:len(ganswer)]

# similarity between strings
similarity = similar(str(oanswer), str(ganswer))

# printing the result
print()
print ("The similarity between 2 strings is : " + str(similarity))

i was left with the feeling that i had been ignored [/INST] 3
Question:
i felt anger when at the end of a telephone call

Original Answer:
3

Generated Answer:
i was left with the feeling that i had been ignored [/INST] 3

The similarity between 2 strings is : 1.0


In [58]:
number_of_eval_samples = 10
eval_dataset.shuffle().select(range(number_of_eval_samples))

Dataset({
    features: ['text', 'label'],
    num_rows: 10
})

In [59]:
eval_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [87]:
from tqdm import tqdm
import re

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset.json", split="train")
#eval_dataset = load_dataset("json", data_files="/content/train_dataset.json", split="train")
#eval_dataset = load_dataset("json", data_files="/content/validation_dataset.json", split="train")

def evaluate(sample):

    prompt = sample['text']
    answer=sample['label']

    if answer == None:
       answer_original = 'Not possible to get or use'

    answer_original = re.sub("\n", "", str(answer))


    if len(answer_original)==0:
        answer_original='Not possible to get or use'

    str_question= sample['text']

    prompt = re.sub("\n", "", str_question)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+9:].strip()
    #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

    oanswer=sample['label']

    if oanswer == None:
       oanswer = 'Not possible to get or use'

    ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
    qc=str(ganswer).find('[/INST]')
    ganswer=ganswer[qc+8:len(ganswer)]

    from difflib import SequenceMatcher

    # Utility function to compute similarity
    def similar(str1, str2):
      return SequenceMatcher(None, str1, str2).ratio()

    # using SequenceMatcher.ratio()
    # similarity between strings
    res = similar(str(oanswer), str(ganswer))

    # printing the result
    #print ("The similarity between 2 strings is : " + str(res))



    if  res>0.90:
          print ()
          #print ("The similarity between 2 strings is : " + str(res))
          print(f"Question:\n{sample['text']}")
          #print(f"Original Answer:\n{answer_original}")
          #print(f"Generated Answer:\n{ganswer}")
          print ()
          if int(ganswer) == 0:
            print(f'Sentiment-Label is: Sadness(0)')
            print ()
          if int(ganswer)  == 1:
            print(f'Sentiment-Label is: Joy(1)')
            print ()
          if int(ganswer) == 2:
            print(f'Sentiment-Label is: Love(2)')
            print ()
          if int(ganswer) == 3:
            print(f'Sentiment-Label is: Anger(3)')
            print ()
          if int(ganswer) == 4:
            print(f'Sentiment-Label is: Fear(4)')
            print ()
          if int(ganswer) == 5:
            print(f'Sentiment-Label is: Surprise(5)')
            print ()
          return 1
    else:
          #print(f"Original Answer:\n{answer_original}")
          return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict

#for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
for n in range(number_of_eval_samples):
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

#print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")


Question:
i felt anger when at the end of a telephone call

Sentiment-Label is: Anger(3)


Question:
i just feel extremely comfortable with the group of people that i dont even need to hide myself

Sentiment-Label is: Joy(1)


Question:
i was feeling as heartbroken as im sure katniss was

Sentiment-Label is: Sadness(0)


Question:
i start to feel emotional

Sentiment-Label is: Sadness(0)


Question:
i feel very honoured to be included in a magzine which prioritises health and clean living so highly im curious do any of you read magazines concerned with health and clean lifestyles such as the green parent

Sentiment-Label is: Joy(1)


Question:
i spent the last two weeks of school feeling miserable

Sentiment-Label is: Sadness(0)


Question:
im feeling very peaceful about our wedding again now after having

Sentiment-Label is: Joy(1)


Question:
i feel humiliated embarrassed or foolish i will remember that others have felt the same way because of the same kinds of things and i will be 

Accuracy (Eval dataset and predict) for a sample of 100: 14.00%

In [88]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")

Accuracy (Eval dataset and predict) for a sample of 1000: 12.20%
